# Постановка задачи

### Задача

- Парсинг данных по книгам с сайта 'https://www.litres.ru/popular/' и построение модели пресказания потенциального рейтинга книги по ее текстовому описанию

# 3. NLP

In [1]:
%matplotlib inline
! pip install pymorphy2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")


[notice] A new release of pip available: 22.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk   # Natural Language Toolkit

## 3.1 Загрузим данные

Загрузим наши обработанные данные

In [3]:
df = pd.read_pickle('../data/df_prepeared.pkl')
display(df.head())
display(df.info())

,Наименование,Краткое описание,Рейтинг,Количество оценок,Дата публикации,Количество отзывов,page_link,new_raiting
0,"Теория невероятности. Как мечтать, чтобы сбыва...","Никакой магии. Только здравый смысл, психологи...",4.8,1541.0,2019-02-16,269.0,https://www.litres.ru/book/tatyana-muzhickaya/...,2.113456
1,"Читайте людей как книгу. Как анализировать, по...","В этой книге Патрика Кинга, автора мировых бес...",4.2,456.0,2021-07-15,49.0,https://www.litres.ru/audiobook/patrik-king/ch...,1.709554
4,Перестаньте угождать людям. Будьте ассертивным...,Угодничество не зря называют болезнью. Оно мож...,4.3,160.0,2022-09-08,32.0,https://www.litres.ru/book/patrik-king/peresta...,1.646717
6,Родная кровь,"В глубине штата Мэн, на берегу залива Атлантич...",4.8,8923.0,2022-01-12,639.0,https://www.litres.ru/book/anne-dar/rodnaya-kr...,3.392183
7,Внутри убийцы,"Чтобы поймать убийцу, нужно смотреть глазами у...",4.7,20230.0,2019-11-05,18000.0,https://www.litres.ru/book/mayk-omer/vnutri-ub...,3.410362


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1152 entries, 0 to 1372
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Наименование        1152 non-null   object        
 1   Краткое описание    1152 non-null   object        
 2   Рейтинг             1152 non-null   float64       
 3   Количество оценок   1152 non-null   float64       
 4   Дата публикации     1152 non-null   datetime64[ns]
 5   Количество отзывов  1152 non-null   float64       
 6   page_link           1152 non-null   object        
 7   new_raiting         1152 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 81.0+ KB


None

Отлично данные загружены, форматы подходят к обработке

## 3.2 Шаг 1. Токенизация

In [4]:
word_tokenizer = nltk.WordPunctTokenizer()

Проведем токенизацию на содержании признака **Краткое описание** первой записи

In [5]:
df.iloc[[3]]['Краткое описание'].values[0]

'В глубине штата Мэн, на берегу залива Атлантического океана, живёт провинциальной жизнью большой, но уединённый город Роар, в котором на протяжении трёх десятилетий орудует серийный убийца, носящий прозвище Больничный Стрелок. На его счету пять жертв, и он вышел на охоту за шестой, однако на его пути внезапно возникают призраки его прошлых кровавых деяний — поколение людей, чьи судьбы затронули его зверства. Получив в своё распоряжение достойных противников, Стрелок произведёт свой самый виртуозный выстрел, вопрос лишь в том, в кого именно попадёт его задержавшаяся во времени пуля.Три части под одной обложкой!'

Сформируем свой (дополнительный) списко стоп-слов из всех имеющихся записей

In [6]:
stop_df = pd.DataFrame()
for i in range(df.shape[0]):
    tokens = word_tokenizer.tokenize(df.iloc[[i]]['Краткое описание'].values[0])
    
    df_tokens = pd.DataFrame(tokens)
    df_tokens['len'] = df_tokens[0].apply(lambda x: len(x))
    
    
    stop_df = pd.concat([stop_df, df_tokens[df_tokens['len']<4][0]], ignore_index=True)

my_stop_w = list(stop_df[0].apply(lambda x: x.lower()))

In [7]:
display(my_stop_w[:10])
display(my_stop_w[-10:])

['.', ',', 'и', '-', 'в', '.', '–', ',', 'и', '–']

['вот', '-', 'вот', 'с', '.', ',', 'и', 'без', 'еще', '.']

## 3.3 Шаг 2. Удаляем стоп-слова

Сначала загрузим список стоп-слов из словаря

In [8]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# примеры стоп-слов
print(len(stop_words))
print(stop_words[:10])

151
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KPS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import re
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

In [10]:
import re
from tqdm import tqdm_notebook
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
all_stop_words = stop_words + my_stop_w

def process_data(df):
    
    data = df.copy()
    texts = []
    targets = []

    # поочередно проходим по всем описаниям
        
    for i in tqdm_notebook(range(data.shape[0])):
        text = data.iloc[[i]]['Краткое описание'].values[0]

        text_lower = words_only(text) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in all_stop_words and not word.isnumeric())]

        data['Краткое описание'].iloc[[i]] = data['Краткое описание'].iloc[[i]].apply(lambda x: tokens) # добавляем в предобработанный список

    return data

In [11]:
# запускаем нашу предобработку
df_tokened = process_data(df)

  0%|          | 0/1152 [00:00<?, ?it/s]

In [12]:
df_tokened

,Наименование,Краткое описание,Рейтинг,Количество оценок,Дата публикации,Количество отзывов,page_link,new_raiting
0,"Теория невероятности. Как мечтать, чтобы сбыва...","[никакой, магии, здравый, смысл, психология, в...",4.8,1541.0,2019-02-16,269.0,https://www.litres.ru/book/tatyana-muzhickaya/...,2.113456
1,"Читайте людей как книгу. Как анализировать, по...","[книге, патрика, кинга, автора, мировых, бестс...",4.2,456.0,2021-07-15,49.0,https://www.litres.ru/audiobook/patrik-king/ch...,1.709554
4,Перестаньте угождать людям. Будьте ассертивным...,"[угодничество, называют, болезнью, искалечить,...",4.3,160.0,2022-09-08,32.0,https://www.litres.ru/book/patrik-king/peresta...,1.646717
6,Родная кровь,"[глубине, штата, берегу, залива, атлантическог...",4.8,8923.0,2022-01-12,639.0,https://www.litres.ru/book/anne-dar/rodnaya-kr...,3.392183
7,Внутри убийцы,"[поймать, убийцу, нужно, смотреть, глазами, уб...",4.7,20230.0,2019-11-05,18000.0,https://www.litres.ru/book/mayk-omer/vnutri-ub...,3.410362
...,...,...,...,...,...,...,...,...
1368,Ящик Скиннера,"[профайлер, окончания, университета, поступает...",4.9,740.0,2023-07-13,21.0,https://www.litres.ru/book/ley-mi/yaschik-skin...,2.672528
1369,Рисунок. Основы учебного академического рисунка,"[предлагаемой, книге, рассматриваются, теорети...",4.4,202.0,2017-08-23,12.0,https://www.litres.ru/book/nikolay-li-10842550...,1.456602
1370,Договориться не проблема. Как добиваться своег...,"[переговорные, техники, описанные, крисом, вос...",4.7,680.0,2017-12-20,75.0,https://www.litres.ru/book/kris-voss/dogovorit...,1.803074
1371,Повелитель мух,"[повелитель, подлинный, шедевр, мировой, литер...",4.7,939.0,2022-05-23,137.0,https://www.litres.ru/audiobook/uilyam-golding...,2.246963


Сравним результаты до обработки и после для конкретного наблюдения:

In [16]:
print('Краткое описание до обработки:')
print('-----------------------------')
display(df['Краткое описание'][3])
print('Краткое описание после обработки:')
print('-----------------------------')
display(df_tokened['Краткое описание'][3])

Краткое описание до обработки:
-----------------------------


'В глубине штата Мэн, на берегу залива Атлантического океана, живёт провинциальной жизнью большой, но уединённый город Роар, в котором на протяжении трёх десятилетий орудует серийный убийца, носящий прозвище Больничный Стрелок. На его счету пять жертв, и он вышел на охоту за шестой, однако на его пути внезапно возникают призраки его прошлых кровавых деяний — поколение людей, чьи судьбы затронули его зверства. Получив в своё распоряжение достойных противников, Стрелок произведёт свой самый виртуозный выстрел, вопрос лишь в том, в кого именно попадёт его задержавшаяся во времени пуля.Три части под одной обложкой!'

Краткое описание после обработки:
-----------------------------


['глубине',
 'штата',
 'берегу',
 'залива',
 'атлантического',
 'океана',
 'живёт',
 'провинциальной',
 'жизнью',
 'большой',
 'уединённый',
 'город',
 'роар',
 'котором',
 'протяжении',
 'трёх',
 'десятилетий',
 'орудует',
 'серийный',
 'убийца',
 'носящий',
 'прозвище',
 'больничный',
 'стрелок',
 'счету',
 'пять',
 'жертв',
 'вышел',
 'охоту',
 'шестой',
 'однако',
 'пути',
 'внезапно',
 'возникают',
 'призраки',
 'прошлых',
 'кровавых',
 'деяний',
 'поколение',
 'людей',
 'судьбы',
 'затронули',
 'зверства',
 'получив',
 'своё',
 'распоряжение',
 'достойных',
 'противников',
 'стрелок',
 'произведёт',
 'свой',
 'самый',
 'виртуозный',
 'выстрел',
 'вопрос',
 'лишь',
 'кого',
 'именно',
 'попадёт',
 'задержавшаяся',
 'времени',
 'пуля',
 'части',
 'одной',
 'обложкой']

Видим, что:
- **Краткое описание** представлено списком слов. 
- Все слова с маленькой буквы. 
- Пунктуация и стоп-слова удалены.

## 3.4 Шаг 3 . Нормализация слов

- Существует 2 наиболее известных способа нормализации слов: **стемминг** и **лемматизация**.
- В общих чертах они похоже, но между этими методами есть различия. В зависимости от языка и задачи тот или иной метод может быть предпочтительнее.

Поэтому создадим два варианта нормализации и выберем тот, что дает наилучший результат на тесте

### 3.4.1 Стемминг

In [17]:
from nltk.stem.snowball import SnowballStemmer

In [18]:
def stemming (list_of_words):
    stemmer = SnowballStemmer("russian")
    new_list = []
    for aword in list_of_words:
        new_list.append(stemmer.stem(aword))
    text = ' '.join(new_list)
    return text

In [19]:
df_tokened['Стемминг'] = df_tokened['Краткое описание'].apply(lambda x: stemming(x))

Сравним результаты до обработки и после для конкретного наблюдения:

In [20]:
print('Краткое описание до обработки:')
print('-----------------------------')
display(df['Краткое описание'][3])
print('Краткое описание после обработки:')
print('-----------------------------')
display(df_tokened['Стемминг'][3])

Краткое описание до обработки:
-----------------------------


'В глубине штата Мэн, на берегу залива Атлантического океана, живёт провинциальной жизнью большой, но уединённый город Роар, в котором на протяжении трёх десятилетий орудует серийный убийца, носящий прозвище Больничный Стрелок. На его счету пять жертв, и он вышел на охоту за шестой, однако на его пути внезапно возникают призраки его прошлых кровавых деяний — поколение людей, чьи судьбы затронули его зверства. Получив в своё распоряжение достойных противников, Стрелок произведёт свой самый виртуозный выстрел, вопрос лишь в том, в кого именно попадёт его задержавшаяся во времени пуля.Три части под одной обложкой!'

Краткое описание после обработки:
-----------------------------


'глубин штат берег залив атлантическ океа живет провинциальн жизн больш уединен город роар котор протяжен трех десятилет оруд серийн убийц нося прозвищ больничн стрелок счет пят жертв вышел охот шест однак пут внезапн возника призрак прошл кровав деян поколен люд судьб затронул зверств получ сво распоряжен достойн противник стрелок произведет сво сам виртуозн выстрел вопрос лиш ког имен попадет задержа времен пул част одн обложк'

### 3.4.2 Лемматизация

In [21]:
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

In [22]:
morph = pymorphy2.MorphAnalyzer()
def lemmatization (list_of_words):
    new_list = []
    for aword in list_of_words:
        new_list.append(morph.parse(aword)[0].normal_form)
    text = ' '.join(new_list)
    return text

In [23]:
df_tokened['Лемматизация'] = df_tokened['Краткое описание'].apply(lambda x: lemmatization(x))

In [24]:
print('Краткое описание до обработки:')
print('-----------------------------')
display(df['Краткое описание'][3])
print('Краткое описание после Стемминга:')
print('-----------------------------')
display(df_tokened['Стемминг'][3])
print('Краткое описание после Лемматизации:')
print('-----------------------------')
display(df_tokened['Лемматизация'][3])

Краткое описание до обработки:
-----------------------------


'В глубине штата Мэн, на берегу залива Атлантического океана, живёт провинциальной жизнью большой, но уединённый город Роар, в котором на протяжении трёх десятилетий орудует серийный убийца, носящий прозвище Больничный Стрелок. На его счету пять жертв, и он вышел на охоту за шестой, однако на его пути внезапно возникают призраки его прошлых кровавых деяний — поколение людей, чьи судьбы затронули его зверства. Получив в своё распоряжение достойных противников, Стрелок произведёт свой самый виртуозный выстрел, вопрос лишь в том, в кого именно попадёт его задержавшаяся во времени пуля.Три части под одной обложкой!'

Краткое описание после Стемминга:
-----------------------------


'глубин штат берег залив атлантическ океа живет провинциальн жизн больш уединен город роар котор протяжен трех десятилет оруд серийн убийц нося прозвищ больничн стрелок счет пят жертв вышел охот шест однак пут внезапн возника призрак прошл кровав деян поколен люд судьб затронул зверств получ сво распоряжен достойн противник стрелок произведет сво сам виртуозн выстрел вопрос лиш ког имен попадет задержа времен пул част одн обложк'

Краткое описание после Лемматизации:
-----------------------------


'глубина штат берег залив атлантический океан жить провинциальный жизнь большой уединённый город роар который протяжение три десятилетие орудовать серийный убийца носить прозвище больничный стрелка счёт пять жертва выйти охота шестой однако путь внезапно возникать призрак прошлый кровавый деяние поколение человек судьба затронуть зверство получить свой распоряжение достойный противник стрелка произвести свой самый виртуозный выстрел вопрос лишь кто именно попасть задержаться время пуля часть один обложка'

Сохраним сформированный датасет

In [26]:
df_tokened.to_pickle('../data/df_NLP.pkl')